# 多个function call并列情况下是否能正常工作 - 改进版本

改进方面：

- 使用了utils2.py - 便于定义不同的模型

初步结论：

- function call并列是可行的，主要是需要编写好“互斥”的提示词

## 准备

In [1]:
%%time

# 加载llm和embeddings
%run ../utils2.py

CPU times: user 2.68 s, sys: 415 ms, total: 3.09 s
Wall time: 2.73 s


In [2]:
%%time

from llama_index.core import Settings

Settings.llm=get_llm()
Settings.embed_model=get_llm()

CPU times: user 670 ms, sys: 19.8 ms, total: 690 ms
Wall time: 689 ms


In [9]:
%%time

from llama_index.core.tools import FunctionTool

def get_news(url: str) -> str:
    """根据url获取新闻内容, 提示词中应提供url"""
    return f"新闻内容来源于{url}"

get_news_tool = FunctionTool.from_defaults(
            fn=get_news)

def query(query: str) ->str:
    """根据提示词查询信息，提示词不应存在url"""

    content='查询到的信息: 1. xxx, 2. xxx ...'
    return f"根据{query}查询到信息\n\n{content}"

query_tool = FunctionTool.from_defaults(
            fn=query)

CPU times: user 635 µs, sys: 0 ns, total: 635 µs
Wall time: 639 µs


In [10]:
%%time

from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
            [get_news_tool,query_tool], verbose=True)

CPU times: user 167 µs, sys: 27 µs, total: 194 µs
Wall time: 195 µs


## 使用

### 输入url的情况

In [16]:
%%time

response = agent.stream_chat("https://www.guancha.cn/internation/2024_06_19_738488.shtml",chat_history=[])
response.print_response_stream()

Thought: The user has provided a URL to a news article. I need to use the 'get_news' tool to extract the content of this article.
Action: get_news
Action Input: {'url': 'https://www.guancha.cn/internation/2024_06_19_738488.shtml'}
Observation: 新闻内容来源于https://www.guancha.cn/internation/2024_06_19_738488.shtml
 您提供的链接指向的新闻文章标题和内容是什么？请提供详细信息，以便我可以帮助您解答相关问题。CPU times: user 502 ms, sys: 105 ms, total: 607 ms
Wall time: 3.16 s


### 输入文字问题的情况

In [17]:
%%time

response = agent.stream_chat("波音公司为啥深陷危机？",chat_history=[])
response.print_response_stream()

Thought: 用户询问关于波音公司的危机原因，我需要查询相关信息来回答这个问题。
Action: query
Action Input: {'query': '波音公司为何陷入危机'}
Observation: 根据波音公司为何陷入危机查询到信息

查询到的信息: 1. xxx, 2. xxx ...
 波音公司深陷危机的原因包括：1.xxx，2.xxx。具体来说，原因可能涉及产品质量、安全问题、财务状况或市场竞争力等。CPU times: user 631 ms, sys: 121 ms, total: 751 ms
Wall time: 2.28 s
